# PyCuAmpcor - Amplitude Cross-Correlation with GPU  
  
## 1. Introduction  
  
Ampcor (Amplitude cross correlation) in InSAR processing offers an estimate of spatial displacements (offsets) with the feature tracking method. The offsets are in resolutions of pixel or sub-pixel (withc additional oversampling).  
  
In practice, we 

* choose a rectangle window, $R(x,y)$, from the reference image, as well as several windows, $S(x+u, y+v)$, from a search image around the same location but offseted by $(u,v)$;

* perform cross-correlation between the search windows with the reference window, to obtain the normalized correlation surface;

$$ c(u,v) = \frac {\sum_{x,y}[R(x,y)-{\bar R}][S(x+u,y+v)-{\bar S}_{u,v}]}{\sqrt{\text{Var}[R]\text{Var}[S_{u,v}]}}$$

* find the maximum of $c(u,v)$ while its location, $(u_m,v_m)$, provides an estimate of the offset. 

PyCuAmpcor follows the same procedure as the FORTRAN code, ampcor.F, in RIOPAC. In order to optimize for GPU, there are some differences in implementations. We list in the following the detailed procedures. 
 
   
  
## 2. List of Procedures  


### 2.0 Parameters


**Image Parameters**

| PyCuAmpcor           | Notes                     |
| :---                 | :----                     |
| referenceImageName   | The file name of the reference/template image |
| referenceImageHeight | The height of the reference image | 
| referenceImageWidth  | The width of the reference image | 
| secondaryImageName   | The file name of the secondary/search image   |
| secondaryImageHeight | The height of the secondary image | 
| secondaryImageWidth  | The width of the secondary image | 
| grossOffsetImageName | The output file name for gross offsets  | 
| offsetImageName      | The output file name for dense offsets  |
| snrImageName         | The output file name for signal-noise-ratio of the correlation |
| covImageName         | The output file name for variance of the correlation surface |

PyCuAmpcor now uses exclusively the GDAL driver to read images, only single-precision binary data are supported. (Image heights/widths are still required as inputs; they are mainly for dimension checking.  We will update later to read them with the GDAL driver). Multi-band is not currently supported, but can be added if desired.

The offset output is arranged in BIP format, with each pixel (azimuth offset, range offset). In addition to a static gross offset (i.e., a constant for all search windows), PyCuAmpcor supports varying gross offsets as inputs (e.g., for glaciers, users can compute the gross offsets with the velocity model for different locations and use them as inputs for PyCuAmpcor. See 2.1 for details. 

The offsetImage only ouputs the (dense) offset values computed from the cross-correlations. Users need to add offsetImage and grossOffsetImage to obtain the total offsets. 


The dimension/direction names used in PyCuAmpcor are:
* the inner-most dimension x(i): row, height, down, azimuth, along the track.
* the outer-most dimension y(j): column, width, across, range, along the sight.
* C/C++/Python use row-major indexing: a[i][j] -> a[i*WIDTH+j]
* FORTRAN/BLAS/CUBLAS use column-major indexing: a[i][j]->a[i+j*LENGTH]

Note that ampcor.F in general uses y for rows and x for columns which is opposite to PyCuAmpcor.

Note also PyCuAmpcor parameters refer to the names used by the PyCuAmpcor Python class. They may be different from those used in C/C++/CUDA, or the cuDenseOffsets.py args. 

**Process Parameters**

| PyCuAmpcor           | Notes                     |
| :---                 | :----                     |
| devID                | The CUDA GPU to be used for computation, usually=0, or users can use the CUDA_VISIBLE_DEVICES=n enviromental variable to choose GPU |
| nStreams | The number of CUDA streams to be used, recommended=2, to overlap the CUDA kernels with data copying, more streams require more memory which isn't alway better | 
| useMmap              | Whether to use memory map cached file I/O, recommended=1, supported by GDAL vrt driver (needs >=3.1.0) and GeoTIFF |
| mmapSize             | The cache size used for memory map, in units of GB. The larger the better, but not exceed 1/4 the total physical memory. |
| numberWindowDownInChunk |  The number of windows processed in a batch/chunk, along lines |  
| numberWindowAcrossInChunk | The number of windows processed in a batch/chunk, along columns | 

Many windows are processed together to maximize the usage of GPU cores; which is called as a Chunk. The total number of windows in a chunk is limited by the GPU memory. We recommend 
numberWindowDownInChunk=1, numberWindowAcrossInChunk=10, for a window size=64.


### 2.1 Iteration over the whole images

The procedures in subsequent sections apply to one pair of reference/secondary windows. In general, many pairs are chosen among the image at difference locations, e.g., arranged with a skip of certain pixels along lines/columns. 

The C/C++/CUDA program accepts inputs with the total number of windows (numberWindowDown, numberWindowAcross) and the starting pixels of each reference window. The purpose is to establish multiple-threads/streams processing. Therefore, users are required to provide/compute these inputs, with tools available from PyCuAmpcor python class. The cuDenseOffsets.py script also does the work. 

We provide some examples below, assuming a PyCuAmpcor class object is created as 

```python
    from isce.contrib.PyCuAmpcor.PyCuAmpcor import PyCuAmpcor
    objOffset = PyCuAmpcor()
```

**To compute the total number of windows**

We use the line direction as an example, assuming parameters as 


```
   margin # the number of pixels to neglect at edges
   halfSearchRangeDown # the half of the search range
   windowSizeHeight # the size of the reference window for feature tracking
   skipSampleDown # the skip in pixels between two reference windows
   referenceImageHeight # the reference image height, usually the same as the secondary image height
```

and the number of windows may be computed along lines as

```python
   objOffset.numberWindowDown = (referenceImageHeight-2*margin-2*halfSearchRangeDown-windowSizeHeight) // skipSampleDown
```

If there is a gross offset, you may also need to subtract that when computing the number of windows. 

The output offset fields will be of size (numberWindowDown, numberWindowAcross). The total number of windows numberWindows = numberWindowDown\*numberWindowAcross.



**To compute the starting pixels of reference/secondary windows**

The starting pixel for the first reference window is usually set as 

```python
   objOffset.referenceStartPixelDownStatic = margin + halfSearchRangeDown
   objOffset.referenceStartPixelAcrossStatic = margin + halfSearchRangeAcross
```

you may also choose other values, e.g., for a particular region of the image, or a certain location for debug purposes.  


With a constant gross offset, call 

```python
   objOffset.setConstantGrossOffset(grossOffsetDown, grossOffsetAcross)
```

to set the starting pixels of all reference and secondary windows. 

The starting pixel for the seconday window will be (referenceStartPixelDownStatic-halfSearchRangeDown+grossOffsetDown, referenceStartPixelAcrossStatic-halfSearchRangeAcross+grossOffsetAcross). 

For cases you choose a varying grossOffset, you may use two numpy arrays to pass the information to PyCuAmpcor, e.g., 

```python
    objOffset.referenceStartPixelDownStatic = objOffset.halfSearchRangeDown + margin
    objOffset.referenceStartPixelAcrossStatic = objOffset.halfSearchRangeAcross + margin
    vD = np.random.randint(0, 10, size =objOffset.numberWindows, dtype=np.int32)
    vA = np.random.randint(0, 1, size = objOffset.numberWindows, dtype=np.int32)
    objOffset.setVaryingGrossOffset(vD, vA)
```    

to set all the starting pixels for reference/secondary windows. 

Sometimes, adding a large gross offset may cause the windows near the edge to be out of range of the orignal image. To avoid memory access errors, call 

```python
   objOffset.checkPixelInImageRange()
```

to verify. If an out-of-range error is reported, you may consider to increase the margin or reduce the number of windows. 



**Parameters**


| PyCuAmpcor           | Notes    |
| :---                 | :----                     |
| skipSampleDown       | The skip in pixels for neighboring windows along height |
| skipSampleAcross     | The skip in pixels for neighboring windows along width |
| numberWindowDown     | the number of windows along height |
| numberWindowAcross   | the number of windows along width  |
| referenceStartPixelDownStatic | the starting pixel location of the first reference window - along height component |
|referenceStartPixelAcrossStatic | the starting pixel location of the first reference window - along width component |





### 2.2 Read a window from Reference/Secondary images

* Load a window of size (windowSizeHeight, windowSizeWidth) from a starting pixel from the reference image

* Load a window of size (windowSizeHeight+2\*halfSearchRangeDown, windowSizeWidth+2\*halfSearchRangeAcross) from the secondary image, the starting position is offseted by (-halfSearchRangeDown, -halfSearchRangeAcross) from the starting position of the reference image (may also be offseted by the grossOffset). 

**Parameters**

| PyCuAmpcor          | CUDA variable       | ampcor.F equivalent   | Notes                     |
| :---                | :---                | :----                 | :---                      | 
| windowSizeHeight    | windowSizeHeightRaw | i_wsyi                |Reference window height     |
| windowSizeWidth     | windowSizeWidthRaw  | i_wsxi                |Reference window width      |
| halfSearchRangeDown | halfSearchRangeDownRaw | i_srchy            | half of the search range along lines |
| halfSearchRangeAcross | halfSearchRangeAcrossRaw | i_srchx            | half of the search range along  |
| 



**Difference to ROIPAC**
No major difference


### 2.3 Perform a cross-correlation and obtain an estimated offset in units of pixels

* Take amplitudes (real) of the (complex) signals in reference/secondary windows
* Compute the normalized correlation surface between reference and secondary windows: the resulting correlation surface is of size (2\*halfSearchRangeDown+1, 2\*halfSearchRangeAcross+1); two cross-correlation methods are offered, time domain or frequency domain.
* Find the location of the maximum/peak in correlation surface. 
* Around the peak position, extract a smaller window from the correlation surface for statistics, such SNR, variance. 

This step provides an initial estimate of the offsets, usually with a large search range. In the following, we will zoom in around the peak, and oversample the windows with a smaller search range. 


**Parameters**

| PyCuAmpcor          | CUDA variable       | ampcor.F equivalent   | Notes                     |
| :---                | :---                | :----                 | :---                      | 
| algorithm           | algorithm           | N/A                   |  the cross-correlation computation method 0=Freq 1=time   |
| corrStatWindowSize  | corrStatWindowSize  | 21               | the size of correlation surface around the peak position used for statistics, may be adjusted   |


**Difference to ROIPAC**

* RIOPAC only offers the time-domain algorithm. The frequency-domain algorithm is faster and is set as default in PyCuAmpcor. 
* RIOPAC proceeds from here only for windows with *good* match. To maintain parallelism, PyCuAmpcor proceeds anyway while leaving the *filtering* to users in post processing. 
  

### 2.4 Extract a smaller window from the secondary window for oversampling

* From the secondary window, we extract a smaller window of size (windowSizeHeightRaw+2\*halfZoomWindowSizeRaw, windowSizeWidthRaw+2\*halfZoomWindowSizeRaw) with the center determined by the peak position. If the peak postion, e.g., along height, is OffsetInit (taking values in \[0, 2\*halfSearchRangeDownRaw\]), the starting position to extract will be OffsetInit+halfSearchRangeDownRaw-halfZoomWindowSizeRaw.

**Parameters**

| PyCuAmpcor          | CUDA variable       | ampcor.F equivalent   | Notes                     |
| :---                | :---                | :----                 | :---                      | 
| N/A                 | halfZoomWindowSizeRaw  | i_srchp(p)=4       |  The smaller search range to zoom-in. In PyCuAmpcor, is determined by zoomWindowSize/(2\*rawDataOversamplingFactor)
 
**Difference to ROIPAC**

RIOPAC always extracts the secondary window centering at the correlation surface peak position. If the peak locates near the edge, zeros are padded if the extraction zone exceeds the window range. In PyCuAmpcor, the extraction zone is shifted to guarantee all pixels are in the original window range. 


### 2.5 Oversampling reference and (extracted) secondary windows 

* oversample both the reference and the (extracted) secondary windows by a factor of 2, which is to avoid aliasing in the complex multiplication of the SAR images. The oversampling is performed with FFT (zero padding), same as in RIOPAC. 
* A deramping procedure is in general required for complex signals before oversampling, to shift the band center to 0. The procedure is only designed to remove a systematc linear phase ramp. It doesn't work for InSAR TOPS mode, whose ramp goes quadratic. Instead, the amplitudes are taken before oversampling.
* the amplitudes (real) are then taken for each pixel of the complex signals in reference and secondary windows.

**Parameters**

| PyCuAmpcor          | CUDA variable       | ampcor.F equivalent   | Notes                     |
| :---                | :---                | :----                 | :---                      | 
| rawDataOversamplingFactor | rawDataOversamplingFactor | i_ovs=2   | the oversampling factor for reference and secondary windows, use 2 for InSAR SLCs. | 
| derampMethod        | derampMethod        | 1 or no effect on TOPS | 0=TOPS, 1=deramping (default), else=skip deramping.  


**Difference to ROIPAC**

RIOPAC enlarges both windows to a size which is a power of 2; ideal for FFT. PyCuAmpcor uses their original sizes for FFT. 

RIOPAC always performs deramping with Method 1, to obtain the ramp by averaging the phase difference between neighboring pixels. For TOPS mode, users need to specify 'mag' as the image datatypes such that the amplitudes are taken before oversampling. Therefore, deramping has no effect. In PyCuAmpcor, derampMethod=0 is equivalent to 'mag' datatype, taking amplitudes but skip deramping. derampMethod=1 always performs deramping, no matter the 'complex' or 'real' image datatypes. Use derampMethod=others to skip deramping, but keeping the original datatypes. 

### 2.6 Cross-Correlate the oversampled reference and secondary windows

* cross-correlate the oversampled reference and secondary windows.
* other procedures are needed to obtain the normalized cross-correlation surface, such as calculating and subtracting the mean values.
* the resulting correlation surface is of size (2\*halfZoomWindowSizeRaw\*rawDataOversamplingFactor+1, 2\*halfZoomWindowSizeRaw\*rawDataOversamplingFactor+1). We cut the last row and column to make it an even sequence, or the size 2\*halfZoomWindowSizeRaw\*rawDataOversamplingFactor=zoomWindowSize. 

**Parameters**

| PyCuAmpcor          | CUDA variable       | ampcor.F equivalent   | Notes                     |
| :---                | :---                | :----                 | :---                      | 
| corrSurfaceZoomInWindow | zoomWindowSize  | i_cw   | The size of correlation surface of the (anti-aliasing) oversampled reference/secondary windows, also used to set halfZoomWindowSizeRaw. Set it to 16 to be consistent with RIOPAC. |

**Difference to ROIPAC**

In RIOPAC, an extra resizing step is performed on the correlation surface, from (2\*halfZoomWindowSizeRaw\*rawDataOversamplingFactor+1, 2\*halfZoomWindowSizeRaw\*rawDataOversamplingFactor+1) to (i_cw, i_cw), centered at the peak (in RIOPAC, the peak seeking is incorporated in the correlation module while is seperate in PyCuAmpcor). i_cw is a user configurable variable; it could be smaller or bigger than 2\*i_srchp\*i_ovs+1=17 (fixed), leading to extraction or enlargement by padding 0s. This procedure is not performed in PyCuAmpcor, as it makes little difference in the next oversampling procedure. 

### 2.7 Oversample the correlation surface and find the peak position

* oversample the (real) correlation surface by a factor oversamplingFactor, or the resulting surface is of size (zoomWindowSize\*oversamplingFactor, zoomWindowSize\*oversamplingFactor) Two oversampling methods are offered, oversamplingMethod=0 (FFT, default), =1(sinc). 
* find the peak position in the oversampled correlation surface, OffsetZoomIn, in range zoomWindowSize\*oversamplingFactor.
* calculate the final offset, from OffsetInit (which is the starting position of secondary window extraction in 2.4),  
  
   offset = (OffsetInit-halfSearchRange)+OffsetZoomIn/(oversamplingFactor\*rawDataOversamplingFactor)
   
Note that this offset does not include the pre-defined gross offset. Users need to add them together if necessay.   
   

**Parameters**

| PyCuAmpcor          | CUDA variable       | ampcor.F equivalent   | Notes                     |
| :---                | :---                | :----                 | :---                      | 
| corrSurfaceOverSamplingFactor | oversamplingFactor  | i_covs   | The oversampling factor for the correlation surface |
| corrSurfaceOverSamplingMethod | oversamplingMethod | i_sinc_fourier=i_sinc | The oversampling method 0=FFT, 1=sinc. |

**Difference to ROIPAC**

RIOPAC by default uses the sinc interpolator (one needs to change the FORTRAN code to use FFT). There is no differnce with the sinc interpolator, while for FFT, RIOPAC always enlarges the window to a power of 2. 




## 3. Examples

### 3.1 


(Additional notes- TBD)
  
## 4. Sinc Interpolator/Oversampler  
  
 
The sinc interpolator formula is defined as   
  
 $$x(t) = \sum_{n=-\infty}^{\infty} x_n f( \Omega_c t-n )$$      
      

with $f(x) = \text{sinc}(x)$ or a complex filter such as the sinc(x) convoluted with Hamming Window used in ampcor. 
 
``` 
   parameter(MAXDECFACTOR=4096) ! maximum lags in interpolation kernels
   r_fintp(0:MAXINTLGH) ! interpolation kernel values  
   i_decfactor = 4096 ! Range migration decimation Factor 
   parameter (MAXINTKERLGH=256) !maximum interpolation kernel length 
   MAXINTLGH=MAXINTKERLGH*MAXDECFACTOR ! maximum interpolation kernel array size 
   i_weight = 1 
   r_pedestal = 0.0 
   r_beta = .75 
   r_relfiltlen = 6.0 
   r_fintp(0:MAXINTLGH)       
```